## Práctica 2: Transformaciones geométricas
Integrantes: David García Díaz y Alejandro Bolaños García

## 1a. Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en tiempo real a través de una interfaz basada en trackbars o equivalente.


In [21]:
import cv2 as cv
import numpy as np

# Variables globales
tx, ty = 0, 0
angle = 0
scale_x = 1.0
scale_y = 1.0
size = (1000, 1000)
image = cv.imread('images/cats.png', cv.IMREAD_COLOR)

# Verificar si la imagen se cargó correctamente
if image is None:
    raise FileNotFoundError("La imagen no se pudo cargar. Verifica la ruta del archivo.")

# Tamaño de la imagen original
h, w = image.shape[:2]

def update_image():
    global tx, ty, angle, scale_x, scale_y

    # Centro de la imagen para la rotación
    center = (w / 2, h / 2)

    # Crear la matriz de transformación afín
    M = cv.getRotationMatrix2D(center, angle, 1.0)
    
    # Ajustar la matriz para incluir el escalado
    M[0, 0] *= scale_x
    M[0, 1] *= scale_x
    M[1, 0] *= scale_y
    M[1, 1] *= scale_y

    # Añadir traslación
    M[0, 2] += tx
    M[1, 2] += ty

    # Aplicar la transformación afín
    transformed_image = cv.warpAffine(image, M, size)
    cv.imshow('Transformaciones', transformed_image)

def on_trackbar_rotation(val):
    global angle
    angle = val
    update_image()

def on_trackbar_translation(val):
    global tx, ty
    tx = val - 100  # Permitir traslación negativa
    ty = val - 100  # Permitir traslación negativa
    update_image()

def on_trackbar_resize(val):
    global scale_x, scale_y
    scale_x = val / 100
    scale_y = val / 100
    update_image()

def on_trackbar_resize_x(val):
    global scale_x
    scale_x = val / 100
    update_image()

def on_trackbar_resize_y(val):
    global scale_y
    scale_y = val / 100
    update_image()

# Crear ventana y barras deslizantes
cv.namedWindow('Transformaciones')
cv.createTrackbar('Rotation', 'Transformaciones', 0, 360, on_trackbar_rotation)
cv.createTrackbar('Translation', 'Transformaciones', 100, 200, on_trackbar_translation)  # Centrar en 0
cv.createTrackbar('Resize', 'Transformaciones', 100, 200, on_trackbar_resize)
cv.createTrackbar('Resize X', 'Transformaciones', 100, 200, on_trackbar_resize_x)
cv.createTrackbar('Resize Y', 'Transformaciones', 100, 200, on_trackbar_resize_y)

# Inicializar la imagen con valores predeterminados
update_image()

cv.waitKey(0)
cv.destroyAllWindows()


## 1b. Dada una imagen trazar una ventana de proyección y proyectar la imagen.

In [19]:
import cv2 as cv
import numpy as np

# Cargar la imagen
image = cv.imread('images/cats.png', cv.IMREAD_COLOR)

# Verificar si la imagen se cargó correctamente
if image is None:
    raise FileNotFoundError("La imagen no se pudo cargar. Verifica la ruta del archivo.")

# Tamaño de la imagen original
h, w = image.shape[:2]

# Definir los puntos de la ventana de proyección en la imagen original (cuatro esquinas)
src_points = np.float32([[0, 0], [w, 0], [w, h], [0, h]])

# Lista para almacenar los puntos de destino
dst_points = []

# Copia de la imagen para dibujar los puntos seleccionados
image_copy = image.copy()

# Función para manejar los eventos del mouse
def mouse(event, x, y, flags, param):
    global dst_points, image

    if event == cv.EVENT_LBUTTONDOWN:
        if len(dst_points) < 4:
            # Almacenar el punto seleccionado
            dst_points.append([x, y])
            # Dibujar un círculo en la posición seleccionada
            cv.circle(image_copy, (x, y), 10, (0, 255, 0), -1)
            cv.imshow('Imagen original', image_copy)

        # Si se seleccionaron 4 puntos, aplicar la transformación
        if len(dst_points) == 4:
            dst_points_np = np.float32(dst_points)

            # Calcular la matriz de transformación de perspectiva
            M = cv.getPerspectiveTransform(src_points, dst_points_np)

            # Aplicar la transformación de perspectiva
            projected_image = cv.warpPerspective(image, M, (w, h))

            # Mostrar la imagen proyectada
            cv.imshow('Imagen Proyectada', projected_image)

# Mostrar la imagen y configurar la función de callback del mouse
cv.imshow('Imagen original', image)
cv.setMouseCallback('Imagen original', mouse)

cv.waitKey(0)
cv.destroyAllWindows()


## 1c. Desarrollar una aplicación que lleve a cabo distorsiones de la lente. Para ello los coeficientes de distorsión deben gobernarse a través de una interfaz

In [22]:
import cv2 as cv
import numpy as np

# Cargar la imagen
image = cv.imread('images/cats.png', cv.IMREAD_COLOR)

# Verificar si la imagen se cargó correctamente
if image is None:
    raise FileNotFoundError("La imagen no se pudo cargar. Verifica la ruta del archivo.")

# Tamaño de la imagen
h, w = image.shape[:2]

# Definir los coeficientes iniciales de distorsión
dist_coeffs = np.zeros((5, 1), np.float32)  # [k1, k2, p1, p2, k3]

# Matriz de la cámara (suponiendo una matriz de identidad)
camera_matrix = np.array([[w, 0, w / 2],
                          [0, h, h / 2],
                          [0, 0, 1]], dtype=np.float32)

# Función para actualizar la imagen distorsionada
def update_image():
    global dist_coeffs, camera_matrix, image
    
    # Aplicar la distorsión usando los coeficientes actuales
    distorted_image = cv.undistort(image, camera_matrix, dist_coeffs)
    
    # Mostrar la imagen distorsionada
    cv.imshow('Distorsion de la Lente', distorted_image)

# Funciones para los eventos de los trackbars
def on_trackbar_k1(val):
    dist_coeffs[0, 0] = val / 1000.0  # Ajustar el valor del coeficiente k1
    update_image()

def on_trackbar_k2(val):
    dist_coeffs[1, 0] = val / 1000.0  # Ajustar el valor del coeficiente k2
    update_image()

def on_trackbar_p1(val):
    dist_coeffs[2, 0] = val / 1000.0  # Ajustar el valor del coeficiente p1
    update_image()

def on_trackbar_p2(val):
    dist_coeffs[3, 0] = val / 1000.0  # Ajustar el valor del coeficiente p2
    update_image()

def on_trackbar_k3(val):
    dist_coeffs[4, 0] = val / 1000.0  # Ajustar el valor del coeficiente k3
    update_image()

# Crear una ventana para mostrar la imagen
cv.namedWindow('Distorsion de la Lente')

# Crear los trackbars para ajustar los coeficientes de distorsión
cv.createTrackbar('K1', 'Distorsion de la Lente', 0, 1000, on_trackbar_k1)
cv.createTrackbar('K2', 'Distorsion de la Lente', 0, 1000, on_trackbar_k2)
cv.createTrackbar('P1', 'Distorsion de la Lente', 0, 1000, on_trackbar_p1)
cv.createTrackbar('P2', 'Distorsion de la Lente', 0, 1000, on_trackbar_p2)
cv.createTrackbar('K3', 'Distorsion de la Lente', 0, 1000, on_trackbar_k3)

update_image()

cv.waitKey(0)
cv.destroyAllWindows()
